In [1]:
import pandas as pd

## MODEL TRAINING

In [2]:
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
#Independent and Dependent Features
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [5]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [6]:
Y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [7]:
#Define which coloums should be oridnal-encoded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [8]:
#Define the custom ranking for each ordinal variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import StandardScaler #Handling Feature scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal encoding
#Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
# Pipeline--> Pipeline functionality used to automate the process by combining multiple libraries
# Numerical Pipeline
num_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='median')),
           ('scaler',StandardScaler())
    ]

)

#Categorical pipeline

cat_pipeline=Pipeline(
    steps=[('imputer',SimpleImputer(strategy='most_frequent')),
           ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
           ('scaler',StandardScaler())


           ]
)

preprocessor=ColumnTransformer([
     ('num_pipeline',num_pipeline,numerical_cols),
     ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [11]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)


In [12]:
preprocessor.fit_transform(X_train)

array([[ 0.44694862,  0.816594  , -1.16172723, ...,  0.87433756,
         0.30390186,  1.34267687],
       [-0.63007986,  0.35803663,  0.39545322, ..., -0.13278771,
        -0.93355801,  0.01266139],
       [ 0.46848919,  0.816594  ,  1.43357352, ..., -0.13278771,
        -0.31482808, -0.65234636],
       ...,
       [ 0.55465147, -0.83421252, -0.64266708, ...,  0.87433756,
        -0.31482808, -1.3173541 ],
       [-1.03935068,  1.1834399 , -0.64266708, ..., -1.13991297,
        -0.31482808, -0.65234636],
       [ 0.51157033,  0.35803663, -0.12360693, ...,  0.87433756,
        -1.55228795,  0.67766913]])

In [13]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [14]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.446949,0.816594,-1.161727,0.593610,0.647349,0.703613,0.874338,0.303902,1.342677
1,-0.630080,0.358037,0.395453,-0.619426,-0.591613,-0.575669,-0.132788,-0.933558,0.012661
2,0.468489,0.816594,1.433574,0.611580,0.556914,0.674538,-0.132788,-0.314828,-0.652346
3,-0.457755,0.266325,-0.642667,-0.376819,-0.311264,-0.313998,0.874338,-0.314828,1.342677
4,-0.436215,-0.650790,-0.123607,-0.260008,-0.238916,-0.299460,-1.139913,-0.933558,1.342677


In [31]:
## MODEL TRAINING

from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [32]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [33]:
regression.coef_

array([[ 6472.27424603,   -56.74942096,   -48.85881858, -1901.24342691,
          324.12203829,  -717.34025393,    72.74398932,  -467.35467419,
          665.36716422]])

In [34]:
regression.intercept_

array([4003.73330477])

In [35]:
import numpy as np
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true, predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mean_squared_error(true, predicted))
    r2_square=r2_score(true, predicted)
    return mae, rmse, r2_square

In [37]:
## Train multiple models

models={
  'LinearRegression':LinearRegression(),
  'Lasso':Lasso(),
  'Ridge':Ridge(),
  'ElasticNet':ElasticNet()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)


    #make predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training performance')
    print('RMSE:',rmse)
    print('MAE',mae)
    print('R2 score',r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')



LinearRegression
Model Training performance
RMSE: 1041.9986571306572
MAE 684.5015179011874
R2 score 93.57508589421782


Lasso
Model Training performance
RMSE: 1041.5879123770824
MAE 684.9303865361078
R2 score 93.58015016106748


Ridge
Model Training performance
RMSE: 1041.8972719672317
MAE 684.6253763829208
R2 score 93.57633610557346


ElasticNet
Model Training performance
RMSE: 1545.7839984906698
MAE 1070.1560694010373
R2 score 85.86061167802848


